In [13]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import differential_evolution, shgo, dual_annealing
from scipy.linalg import fractional_matrix_power
import math

In [14]:
# reads in samples from multiple files
def read_in_samples(file_names, X_column="Homo_sapiens", Y_column="Solenodon_paradoxus"):
  #file_names is a list of file names
  sequences = [] #sequences is a list of tuples of pairs of sequences [(x1,y1), (x2, y2)]
  d = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'a': 0, 'c': 1, 'g': 2, 't': 3}
  for file in file_names:
    df = pd.read_table(file, sep=" ")
    df = df.replace({X_column: d, Y_column: d})
    x = df[X_column].to_list()
    y = df[Y_column].to_list()
    sequences.append((x,y))
  return sequences

In [15]:
def overall_log_likelihood(A, sequences):
  # sequences is a list of tuples of pairs of **INDEPENDENT** sequences along with T
  # sequences: [(x1, y1), (x2, y2)...] where each x and y is a list of ints
  A = np.reshape(A, (4,4)) # This is because minimize flattens Arrays automatically 
  sum = 0 
  for sequence in sequences:
    for i in range(len(sequence[0])): 
      a = sequence[0][i]
      b = sequence[1][i]
      #print(a,b,sum)
      sum += np.log(A[a, b])
  return sum

In [16]:
lnc = read_in_samples(['https://raw.githubusercontent.com/joshuamb/ZoonomiaCG/main/lncRNA-batch101/lnc-batch101-final_sequences.txt'])
UTR = read_in_samples(['https://raw.githubusercontent.com/joshuamb/ZoonomiaCG/main/UTR-batch10000/UTR-batch10000-final_sequences.txt'])

In [17]:
initial_guess = np.array([0.25, 0.25, 0.25, 0.25,0.25, 0.25, 0.25, 0.25,0.25, 0.25, 0.25, 0.25,0.25, 0.25, 0.25, 0.25]) 

#constraints that all rows must sum to 1
newOverallLogLikelihood_UTR = lambda A: -overall_log_likelihood(A, UTR)
newOverallLogLikelihood_lnc = lambda A: -overall_log_likelihood(A, lnc)
lb = 1
ub = 1
m = np.array([[1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0], [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1]])
linear_constraint = LinearConstraint(m, 1,1)

# define a constraint that all numbers need to be between 0 and 1 
bounds = Bounds([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])

result_UTR = minimize(fun=newOverallLogLikelihood_UTR, x0=initial_guess, method = 'trust-constr', bounds=bounds, constraints=linear_constraint)
result_lnc = minimize(fun=newOverallLogLikelihood_lnc, x0=initial_guess, method = 'trust-constr', bounds=bounds, constraints=linear_constraint)

In [18]:
print("UTR:")
At_UTR = np.reshape(result_UTR.x, (4,4))
print(At_UTR.round(2))
print("lncRNA:")
At_lnc = np.reshape(result_lnc.x, (4,4))
print(At_lnc.round(2))

UTR:
[[0.76 0.06 0.12 0.06]
 [0.1  0.63 0.06 0.22]
 [0.21 0.06 0.63 0.09]
 [0.07 0.13 0.05 0.75]]
lncRNA:
[[0.76 0.05 0.12 0.07]
 [0.1  0.61 0.05 0.24]
 [0.23 0.05 0.62 0.1 ]
 [0.07 0.12 0.05 0.77]]
